**FEEL FREE TO UPVOTE**  （＾ｖ＾）

In [1]:

#v20提交成绩得到了3.688， cv 成绩3.6519
#使用authorized_flag 进行特征分别处理,收到了奇效
#使用countVectorizer 对类别变量进行编码，内存存在压力

In [2]:
import numpy as np
import pandas as pd
import datetime
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
import warnings
import time
warnings.filterwarnings('ignore')
np.random.seed(4950)

In [3]:
import pickle

In [4]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df_hist_trans = pd.read_csv('../input/historical_transactions.csv')
df_new_merchant_trans = pd.read_csv('../input/new_merchant_transactions.csv')

In [6]:
df_hist_trans = reduce_mem_usage(df_hist_trans)
df_new_merchant_trans = reduce_mem_usage(df_new_merchant_trans)

Mem. usage decreased to 1749.11 Mb (43.7% reduction)
Mem. usage decreased to 114.20 Mb (45.5% reduction)


In [7]:
df_hist_trans['merchant_id'].nunique() 

326311

In [ ]:
#使用countVectorizer对category特征进行处理，别人号称可以提升 千分之3
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(token_pattern='\w{1,}') 

card_id
C_ID_00007093c1    19 33 34 18 8 19 33 19 33 21 21 8 33 33 19 19 ...
C_ID_0001238066    16 33 33 33 19 33 37 33 37 19 37 27 19 19 16 3...
C_ID_0001506ef0    27 33 33 29 21 33 29 33 29 27 27 27 16 33 8 33...
C_ID_0001793786    37 27 16 36 37 25 29 36 37 34 34 25 33 33 27 4...
C_ID_000183fdda    29 27 32 27 16 27 37 1 16 34 25 32 36 21 25 27...
C_ID_00024e244b    34 33 16 34 33 19 34 33 16 16 33 33 16 33 34 3...
C_ID_0002709b5a    7 33 33 33 33 33 19 19 33 33 8 1 16 29 33 13 1...
C_ID_00027503e2    29 33 15 34 33 34 34 37 33 33 33 33 33 15 37 3...
C_ID_000298032a    27 37 29 33 34 16 36 34 36 30 34 29 34 37 33 3...
C_ID_0002ba3c2e    33 33 33 33 19 37 33 33 18 33 34 33 33 25 34 1...
C_ID_0002c7c2c1    15 37 27 29 17 37 33 16 15 15 1 34 29 29 37 29...
C_ID_00032df08f    33 17 33 29 27 29 29 17 21 7 7 29 34 29 1 7 21...
C_ID_0003754056    33 34 33 29 34 29 34 33 33 29 33 34 37 1 34 33...
C_ID_000377f6a0    41 41 37 41 29 29 16 27 37 8 34 27 27 1 37 37 ...
C_ID_0003be3c83    33 29 3

<class 'scipy.sparse.csr.csr_matrix'>
1 merchant_category_id
card_id
C_ID_00007093c1    2 2 2 2 -1 2 2 2 2 2 2 -1 2 2 2 2 -1 2 2 2 2 2...
C_ID_0001238066    15 9 9 9 9 9 9 9 9 9 9 9 20 9 9 20 9 9 9 9 9 9...
C_ID_0001506ef0    19 19 19 19 19 19 19 19 9 19 19 19 19 19 19 19...
C_ID_0001793786    -1 -1 -1 -1 24 -1 11 9 -1 -1 -1 11 24 -1 -1 24...
C_ID_000183fdda    9 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 ...
C_ID_00024e244b    7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 9 7 7 7 ...
C_ID_0002709b5a    18 18 18 18 18 18 18 21 18 18 -1 21 18 18 18 1...
C_ID_00027503e2    9 19 19 19 19 19 19 19 19 19 19 19 19 19 19 19...
C_ID_000298032a    16 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9...
C_ID_0002ba3c2e    18 18 18 18 18 18 18 18 18 18 18 18 18 18 18 1...
C_ID_0002c7c2c1    9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 ...
C_ID_00032df08f    9 -1 9 9 9 -1 9 -1 9 9 -1 -1 9 -1 9 -1 9 9 -1 ...
C_ID_0003754056    15 15 15 9 15 9 15 15 15 15 15 15 15 15 15 15 ...
C_ID_000377f6a0    2 2 2 2 9 2 2 2

In [ ]:
df_hist_trans.purchase_date=pd.to_datetime(df_hist_trans.purchase_date)
pur_date=df_hist_trans[df_hist_trans.month_lag==0].groupby('card_id').purchase_date.max()
card_id_nan_unique=df_hist_trans[df_hist_trans.card_id.isin(pur_date.index)==False].card_id.unique()
df=pd.DataFrame(card_id_nan_unique)
df['month_lag_date']=pd.to_datetime('2018-02') # Seetting all nan values to 2018 Feb
df.set_index(0,inplace=True)
new_map=df.month_lag_date
method=pur_date.append(new_map)
del df,pur_date
gc.collect()
time.sleep(5)

In [ ]:
for df in [df_hist_trans,df_new_merchant_trans]:
#     df['category_2'].fillna(1.0,inplace=True)
#     df['category_3'].fillna('A',inplace=True)
#     df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
    #修改特征
    df['category_2'].fillna(-1,inplace=True)
    df['category_3'].fillna('other',inplace=True)
    df['merchant_id'].fillna('other',inplace=True)
#     df.loc[df['installments'].isin([999,-1]),'installments'] = 0 

In [ ]:
def get_new_columns(name,aggs):
    #for for 写法 nice
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [ ]:
for df in [df_hist_trans,df_new_merchant_trans]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
    df['category_1'] = df['category_1'].map({'Y':1, 'N':0}) 
    #https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion/73244
    ##Think about this part!
    ## This line of code may be missleading since there is no single ref date. 
    df['month_diff'] = ((datetime.datetime(2018,6,1) - df['purchase_date']).dt.days)//30 # original code
    df['month_diff'] += df['month_lag']
    
    ## removing 'month_diff' reduces LB score to 3.682
    ## This line of code can be also missleading since for new merchant hist it might be negative. But why we care about it.
    #df['month_diff'] = ((df['card_id'].map(method) - df['purchase_date']).dt.days)//30 # method that I used in my code
    
    ##But we already have mongth lag for each purchase! So we don't really need it!
    ### month_diff seems to be important feature think more about it
    
    # This makes it even more irrelevant for my case since it will be equal to zero in most of the cases.So, let's remove it for my case
    #df['month_diff'] += df['month_lag']

In [ ]:
(pd.to_datetime('2018-12-30')-pd.to_datetime('2019-12-30')).days//30

In [ ]:
df_new_merchant_trans.head()

In [ ]:
#因为根据auth_flag 将特征分成了两部分，这里聚合一个全局的auth_flag 特征
aggs = {}
aggs['purchase_amount'] = ['sum']
# aggs['installments'] = ['sum','max','min','mean','var','median']
aggs['authorized_flag'] = ['sum', 'mean','std']
aggs['card_id'] = ['size']
auth_flag = df_hist_trans.groupby(['card_id']).agg(aggs)
auth_flag.columns = get_new_columns('auth_flag',aggs)
auth_flag.reset_index(inplace=True)
df_train = df_train.merge(auth_flag,on='card_id',how='left')
df_test = df_test.merge(auth_flag,on='card_id',how='left')
del auth_flag
gc.collect()

In [ ]:
#对 authorized_flag进行结果编码
aggs = {}
for col in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
    df_hist_trans[col+'_auth_mean'] = df_hist_trans.groupby([col])['authorized_flag'].transform('mean')
    df_hist_trans[col+'_auth_sum'] = df_hist_trans.groupby([col])['authorized_flag'].transform('sum') 
    aggs[col+'_auth_mean'] = ['mean']
    aggs[col+'_auth_sum'] = ['sum'] 
auth_encoder = df_hist_trans.groupby(['card_id']).agg(aggs)
auth_encoder.columns = get_new_columns('auth_encoder',aggs)
auth_encoder.reset_index(inplace=True)
df_train = df_train.merge(auth_encoder,on='card_id',how='left')
df_test = df_test.merge(auth_encoder,on='card_id',how='left')
del auth_encoder
gc.collect()

In [ ]:
def aggregate_per_month(prefix,history,agg_func):
    grouped = history.groupby(['card_id', 'month_lag'])
    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = [prefix + '_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)

    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = [prefix + '_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True) 
    return final_group

In [ ]:
#对授权码 进行按月聚合
agg_func = {'authorized_flag': [ 'sum', 'mean','median']}
final_group =  aggregate_per_month('agg_per_month_total',df_hist_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group

In [ ]:
authorized_transactions = df_hist_trans[df_hist_trans['authorized_flag'] == 1]
df_hist_trans = df_hist_trans[df_hist_trans['authorized_flag'] == 0]

In [ ]:
df_hist_trans.shape

In [ ]:
agg_func = {'purchase_amount': [ 'sum', 'mean', 'min', 'max', 'std']}
final_group =  aggregate_per_month('agg_per_month_auth',authorized_transactions,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
final_group =  aggregate_per_month('agg_per_month_hist',df_hist_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
final_group =  aggregate_per_month('agg_per_month_hist_new',df_new_merchant_trans,agg_func) 
df_train = df_train.merge(final_group,on='card_id',how='left')
df_test = df_test.merge(final_group,on='card_id',how='left')
del final_group
gc.collect()

In [ ]:
df_train['outliers'] = 0
df_train.loc[df_train['target'] < -30, 'outliers'] = 1
df_train['outliers'].value_counts()

In [ ]:
i = 0
for df in [authorized_transactions,df_hist_trans]:
    aggs = {}
    for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',
                'state_id','city_id']:
        aggs[col] = ['nunique']

    aggs['purchase_amount'] = ['sum','max','min','mean','var','median']
    aggs['installments'] = ['sum','max','min','mean','var','median']
    aggs['purchase_date'] = ['max','min']
    aggs['month_lag'] = ['max','min','mean','var','median']
    
    ## I also remove this part since it is also irrelevant since it is equivelent to month_lag. But it makes it worser
    aggs['month_diff'] = ['mean','median']



#     aggs['authorized_flag'] = ['sum', 'mean','median']
    aggs['weekend'] = ['sum', 'mean']
    aggs['category_1'] = ['sum', 'mean']
    aggs['card_id'] = ['size']
    #产生交叉特征，内存有问题
    features = ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id'
               ,'merchant_id']
#     for coli in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#         for colj in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#             df[coli + colj] = df[coli].astype(str) + df[colj].astype(str)
#             features.append(coli + colj)
    for col in features:
        df[col+'_mean'] = df.groupby([col])['purchase_amount'].transform('mean')
        df[col+'_sum'] = df.groupby([col])['purchase_amount'].transform('sum') 
        aggs[col+'_mean'] = ['mean']
        aggs[col+'_sum'] = ['sum'] 
#         添加特征，使用outlier进行编码
#         outliers_mean = df.groupby([col])['outliers'].mean()
#         outliers_sum = df.groupby([col])['outliers'].sum()
#         df[col+'_outliers_mean'] = df[col].map(outliers_mean)
#         df[col+'_outliers_sum'] = df[col].map(outliers_sum)
#         aggs[col+'_outliers_mean'] = ['mean']
#         aggs[col+'_outliers_sum'] =['sum']   
        
    if i == 0:
        prefix = 'auth_hist'
    else:
        prefix = 'hist'
    new_columns = get_new_columns(prefix,aggs)
    i += 1
    # df_hist_trans.sort_values(['card_id','purchase_date'],inplace = True)

    df_hist_trans_group = df.groupby('card_id').agg(aggs)
    df_hist_trans_group.columns = new_columns
    df_hist_trans_group.reset_index(drop=False,inplace=True)
    df_hist_trans_group[prefix + '_purchase_date_diff'] = (df_hist_trans_group[prefix + '_purchase_date_max'] - df_hist_trans_group[prefix + '_purchase_date_min']).dt.days
    df_hist_trans_group[prefix + '_purchase_date_average'] = df_hist_trans_group[prefix + '_purchase_date_diff']/df_hist_trans_group[prefix + '_card_id_size']
    ###Think about below line of code  
    ###up to now for history means month_lag_0 to purdate_min
#     df_hist_trans_group[prefix + '_purchase_date_uptonow'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group[prefix + '_purchase_date_max']).dt.days
    ### This is new feature but I change the name to the old name since it makes more sense. Total number of days from first purchase to month lag 0 (until promotion)
    df_hist_trans_group[prefix + '_purchase_date_uptonow'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group[prefix + '_purchase_date_min']).dt.days
    ### This is old feature but I change the name to [prefix + '_first_purchase'] from [prefix + '_purchase_date_uptonow']
    ### since it makes more sense. Number of days for the last purchase from month_lag_0
    df_hist_trans_group[prefix + '_first_purchase'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group[prefix + '_purchase_date_max']).dt.days
    #下面这个特征考虑了：有的人可能购买频率较低，但是还是忠实粉丝的情况
#     df_hist_trans_group[prefix + '_purchase_date_uptonow_ave'] =  df_hist_trans_group[prefix + '_purchase_date_uptonow']/df_hist_trans_group[prefix + '_purchase_date_average']
    #每一个card中未授权消费次数
#     df_hist_trans_group[prefix + '_unauthorized_number'] = df_hist_trans_group[prefix + '_card_id_size'] - df_hist_trans_group[prefix + '_authorized_flag_sum']
    #最近活跃时间，确实是一个强特征，感觉可以再挖出来几个特征，比如最近5次消费时间，最近10次消费时间，如果值比较小，说明最近很活跃
    #没有效果
    # grouped =  df_hist_trans.groupby('card_id')['purchase_date']
    # df_hist_trans_group['hist_purchase_5thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(5)).dt.days
    # df_hist_trans_group['hist_purchase_3thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(3)).dt.days
    # df_hist_trans_group['hist_purchase_10thdate_uptonow'] =  (datetime.datetime.today() -grouped.shift(10)).dt.days
    df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
    df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
    del df_hist_trans_group
    gc.collect()
    time.sleep(5)
    #内存问题，暂时先放弃
    for col in ['subsector_id','merchant_category_id','state_id','city_id']:
        df[col]=df[col].astype(str)
        grouped = df[['card_id',col]].groupby('card_id')[col].aggregate(lambda x: ' '.join(x))
        cv_fit=cv.fit_transform(grouped)
        cv_df = pd.DataFrame(cv_fit.toarray())
        print(i,col)
        if i == 1:
            cv_df.columns = ['auth' + '_' + col + "_" + str(c) for c in cv_df.columns]
        elif i == 2:
             cv_df.columns = ['hist' + '_' + col + "_" + str(c) for c in cv_df.columns]
        cv_df['card_id'] = grouped.index
        df_train = df_train.merge(cv_df,on='card_id',how='left')
        df_test = df_test.merge(cv_df,on='card_id',how='left')
        del cv_df,grouped
        gc.collect()
    del df
    gc.collect()
    time.sleep(5)

In [ ]:
aggs = {}
#添加特征
for col in ['month','hour','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id',
            'state_id','city_id']:
    aggs[col] = ['nunique']
    
aggs['purchase_amount'] = ['sum','max','min','mean','var','median']
aggs['installments'] = ['sum','max','min','mean','var','median']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var','median']

### Now we also need to remove from here since we already have month_lag
aggs['month_diff'] = ['mean','median']


aggs['weekend'] = ['sum', 'mean']
aggs['authorized_flag'] = ['sum', 'mean','median']
aggs['category_1'] = ['sum', 'mean']
aggs['card_id'] = ['size']

#添加特征   
features = ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id'
               ,'merchant_id']
#产生交叉特征，内存有问题
# for coli in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#     for colj in ['category_2','category_3','state_id','subsector_id','merchant_category_id','city_id']:
#         df_new_merchant_trans[coli + colj] = df_new_merchant_trans[coli].astype(str) + df_new_merchant_trans[colj].astype(str)
#         features.append(coli + colj)
for col in features:
    df_new_merchant_trans[col+'_mean'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('mean')
    df_new_merchant_trans[col+'_sum'] = df_new_merchant_trans.groupby([col])['purchase_amount'].transform('sum')
    aggs[col+'_mean'] = ['mean']
    aggs[col+'_sum'] = ['sum']
    #添加特征，使用outlier进行编码
#     outliers_mean = df.groupby([col])['outliers'].mean()
#     outliers_sum = df.groupby([col])['outliers'].sum()
#     df[col+'_outliers_mean'] = df[col].map(outliers_mean)
#     df[col+'_outliers_sum'] = df[col].map(outliers_sum)
#     aggs[col+'_outliers_mean'] = ['mean']
#     aggs[col+'_outliers_sum'] =['sum']   
    
new_columns = get_new_columns('new_hist',aggs)
# df_new_merchant_trans.sort_values(['card_id','purchase_date'],inplace = True)
df_hist_trans_group = df_new_merchant_trans.groupby('card_id').agg(aggs)
df_hist_trans_group.columns = new_columns
df_hist_trans_group.reset_index(drop=False,inplace=True)
df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']


# it is fine for new mech transaction history. This means total number of days from month_lag_0 ti kast purchase
df_hist_trans_group['new_hist_purchase_date_uptonow'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group['new_hist_purchase_date_max']).dt.days

# new feature for the first purchase.This means total number of days from month_lag_0 to first purchase.
df_hist_trans_group['new_hist_first_purchase'] = (df_hist_trans_group['card_id'].map(method) - df_hist_trans_group['new_hist_purchase_date_min']).dt.days

#下面这个特征考虑了：有的人可能购买频率较低，但是还是忠实粉丝的情况
# df_hist_trans_group['new_hist_purchase_date_uptonow_ave'] =  df_hist_trans_group['new_hist_purchase_date_uptonow']/df_hist_trans_group['new_hist_purchase_date_average']

#每一个card中未授权消费次数
df_hist_trans_group['new_hist_unauthorized_number'] = df_hist_trans_group['new_hist_card_id_size'] - df_hist_trans_group['new_hist_authorized_flag_sum']
# grouped = df_new_merchant_trans.groupby('card_id')['purchase_date']
# df_hist_trans_group['new_hist_purchase_5thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(5)).dt.days
# df_hist_trans_group['new_hist_purchase_3thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(3)).dt.days
# df_hist_trans_group['new_hist_purchase_10thdate_uptonow'] =  (datetime.datetime.today() - grouped.shift(10)).dt.days

df_train = df_train.merge(df_hist_trans_group,on='card_id',how='left')
df_test = df_test.merge(df_hist_trans_group,on='card_id',how='left')
del df_hist_trans_group
gc.collect()
del df_new_merchant_trans
gc.collect()
time.sleep(5)

In [ ]:
for df in [df_train,df_test]:
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (df['card_id'].map(method) - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    #添加特征
    df['auth_hist_first_buy'] = (df['auth_hist_purchase_date_min'] - df['first_active_month']).dt.days
    #修改特征
    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max',\
                     'new_hist_purchase_date_min','auth_hist_purchase_date_max','auth_hist_purchase_date_min']:
        df[f] = df[f].astype(np.int64) * 1e-9
    for f in ['auth_encoder_category_2_auth_sum_sum','auth_encoder_category_3_auth_sum_sum',
            'auth_encoder_state_id_auth_sum_sum','auth_encoder_subsector_id_auth_sum_sum',
            'auth_encoder_merchant_category_id_auth_sum_sum','auth_encoder_city_id_auth_sum_sum']:
        df[f] = df[f].astype(np.int64) * 1e-9
    #上面auth_flag已经聚合过了card_id_size ,purchase_amount
#     df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']  + df['auth_hist_card_id_size']
#     df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']+df['auth_hist_purchase_amount_sum']
#添加特征

for f in ['feature_1','feature_2','feature_3','month','dayofweek']:
    order_label1 = df_train.groupby([f])['outliers'].mean()
    df_train[f+'_outliers_mean'] = df_train[f].map(order_label1)
    df_test[f+'_outliers_mean'] = df_test[f].map(order_label1)
    
    order_label2 = df_train.groupby([f])['outliers'].sum()
    df_train[f+'_outliers_sum'] = df_train[f].map(order_label2)
    df_test[f+'_outliers_sum'] = df_test[f].map(order_label2)
    
#     order_label1 = df_train.groupby([f])['target'].mean()
#     df_train[f+'_target_mean'] = df_train[f].map(order_label1)
#     df_test[f+'_target_sum'] = df_test[f].map(order_label1)
#     order_label2 = df_train.gorupby([f])['target'].sum()
#     df_train[f+'_target_sum'] = df_train[f].map(order_label2)
#     df_test[f+'_target_sum'] = df_test[f].map(order_label2)
 
# get_dummies 似乎有一点点不良影响
df_train = pd.get_dummies(df_train,columns =['feature_1','feature_2'])
df_test = pd.get_dummies(df_test,columns =['feature_1','feature_2'])

In [ ]:
#首次购买的时间居然早于首次激活的时间，进行调整
df_train.loc[df_train['auth_hist_first_buy'] < 0, 'auth_hist_first_buy'] = -1
df_train.loc[df_train['hist_first_buy'] < 0, 'hist_first_buy'] = -1
df_train.head()

In [ ]:
exclude_features = []
exclude_features += ['card_id', 'first_active_month','target','outliers']
df_train_columns = [c for c in df_train.columns if c not in exclude_features ]
target = df_train['target']
del df_train['target']

In [ ]:
len(df_train_columns)
len(exclude_features)

## Above is the same with wang, I call it wang's features

### I replace wang's stratified CV with normal lgb.cv

In [ ]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 42,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 48,
         "random_state": 4950}

In [ ]:
trn_data = lgb.Dataset(df_train[df_train_columns], label=target)#, categorical_feature=categorical_feats)

In [ ]:
#trn_data = lgb.Dataset(df_train[df_train_columns], label=target, categorical_feature=['month','dayofweek','weekofyear'])

In [ ]:
lgb_cv = lgb.cv(param, trn_data, 10000, early_stopping_rounds=600, verbose_eval=200, stratified=False)

In [ ]:
lgb_cv

Results without categorical_feature=['month','dayofweek','weekofyear'].
[200]	cv_agg's rmse: 3.69075 + 0.0319068
[400]	cv_agg's rmse: 3.6695 + 0.0304884
[600]	cv_agg's rmse: 3.66173 + 0.0294372
[800]	cv_agg's rmse: 3.65849 + 0.0290465
[1000]	cv_agg's rmse: 3.65738 + 0.0285758
[1200]	cv_agg's rmse: 3.65658 + 0.0280419
[1400]	cv_agg's rmse: 3.65644 + 0.0274495
[1600]	cv_agg's rmse: 3.65649 + 0.0271829
[1800]	cv_agg's rmse: 3.65663 + 0.0268914

In [ ]:
lgb_cv['rmse-mean'].index(min(lgb_cv['rmse-mean']))

In [ ]:
min(lgb_cv['rmse-mean'])

In [ ]:
len(lgb_cv['rmse-mean'])

Original wang results:

[200]	cv_agg's rmse: 3.68161 + 0.0299797
[400]	cv_agg's rmse: 3.66225 + 0.0286057
[600]	cv_agg's rmse: 3.65503 + 0.0277111
[800]	cv_agg's rmse: 3.65259 + 0.0276703
[1000]	cv_agg's rmse: 3.65156 + 0.027532
[1200]	cv_agg's rmse: 3.65109 + 0.0273339
[1400]	cv_agg's rmse: 3.65112 + 0.0268877
[1600]	cv_agg's rmse: 3.65121 + 0.0265736
[1800]	cv_agg's rmse: 3.65127 + 0.0266131

lgb.cv show's that num_boost_round is around 2000, so we train a model with num_boost_round=2000

In [ ]:
clf = lgb.train(param, trn_data, 2000, valid_sets=(trn_data), verbose_eval=200)

Original Wang's results:
[200]	training's rmse: 3.57275
[400]	training's rmse: 3.48472
[600]	training's rmse: 3.42865
[800]	training's rmse: 3.3851
[1000]	training's rmse: 3.34906
[1200]	training's rmse: 3.3155
[1400]	training's rmse: 3.28252
[1600]	training's rmse: 3.25342
[1800]	training's rmse: 3.22531
[2000]	training's rmse: 3.19685

predict and save

In [ ]:
y_pred = clf.predict(df_test[df_train_columns], num_iteration=clf.best_iteration)

pd.DataFrame({
        'card_id': df_test.card_id,
        'target': y_pred
}).to_csv('wang_lgb_cv_pur_date_1_updated', index=False)


### LB: 3.676

## Outlier detection using wang's features

the param is same as regression and just adjust to binary classification

best num_boost_round is 500, so we use 500 to train a classifer

In [ ]:
outlier = np.zeros(len(target), dtype=np.int)

In [ ]:
outlier[target < -33] = 1

In [ ]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 32, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 42,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 48,
         "scale_pos_weight": 15,
         "random_state": 4950}

In [ ]:
trn_data = lgb.Dataset(df_train[df_train_columns], label=outlier)#, categorical_feature=categorical_feats)

In [ ]:
lgb_cv = lgb.cv(param, trn_data, 10000, early_stopping_rounds=600, verbose_eval=200, stratified=False)

In [ ]:
lgb_cv['auc-mean'][-1]

In [ ]:
clf = lgb.train(param, trn_data, 500, verbose_eval=100, valid_sets=(trn_data))

In [ ]:
te_prob = clf.predict(df_test[df_train_columns])

In [ ]:
te_prob.min(), te_prob.max()

In [ ]:
wang_out=pd.read_csv('wang_lgb_cv_pur_date_1_updated')
wang_out['outlier_prob1'] = te_prob

In [ ]:
_40=wang_out.sort_values('outlier_prob1',ascending=False)

In [ ]:
top20 = _40.card_id.values[:20].tolist()

In [ ]:
wang_out.loc[wang_out.card_id.isin(top20), 'target'] = -33.21928095
wang_out[['card_id', 'target']].to_csv('wang_lgb_cv_pur_date_1_updated_top20.csv', index=False)

##LB: 3.671

In [ ]:
top20_50 = _40.card_id.values[20:80].tolist()

In [ ]:
top20_50_df = wang_out[wang_out.card_id.isin(top20_50)]

In [ ]:
top20_50_target = top20_50_df.outlier_prob1 * (-33.21928095)

In [ ]:
wang_out.loc[wang_out.card_id.isin(top20_50), 'target'] = top20_50_target

In [ ]:
wang_out[['card_id', 'target']].to_csv('wang_lgb_cv_pur_date_1_updated_top20_top20-50.csv', index=False)

##LB: 3.668

In [ ]:
wang_out.loc[wang_out.card_id.isin(top20_50)]